### Setup

In [9]:
import torch
import numpy as np
import timm
import torchvision as tvision
import tensorflow as tf
import requests

from PIL import Image
from urllib.request import urlopen
from transformers import AutoImageProcessor, ResNetForImageClassification

from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import (CarliniL2Method, ProjectedGradientDescent, SaliencyMapMethod,
                                 ElasticNet, DeepFool, HopSkipJump)

from cleverhans.torch.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.torch.attacks.carlini_wagner_l2 import carlini_wagner_l2

class DoubleModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model.to(dtype=torch.double).cuda()

    def forward(self, x):
        x = x.to(dtype=torch.double)
        outputs = self.model(x)
        logits = outputs.logits.to(dtype=torch.double)
        return logits
    
class DoubleModelTimmWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model.to(dtype=torch.double)

    def forward(self, x):
        x = x.to(dtype=torch.double)
        outputs = self.model(x)
        return outputs.to(dtype=torch.double)


# Setting up models

mobile_model = timm.create_model('tf_mobilenetv3_large_minimal_100.in1k', pretrained=True)
mobile_model = mobile_model.cuda().eval()

mobile_data_config = timm.data.resolve_model_data_config(mobile_model)
mobile_transforms = timm.data.create_transform(**mobile_data_config, is_training=False)

vgg16_model = timm.create_model('vgg16.tv_in1k', pretrained=True)
vgg16_model = vgg16_model.cuda().eval()

vgg16_data_config = timm.data.resolve_model_data_config(vgg16_model)
vgg16_transforms = timm.data.create_transform(**vgg16_data_config, is_training=False)

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
hf_resnet = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")
resnet_model = DoubleModelWrapper(hf_resnet)


# Downloading labels for ImageNet1k
url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
imagenet_labels = requests.get(url).json()

# prepare image
image = Image.open(urlopen("https://transforms.stlzoo.org/production/animals/red-kangaroo-02-01.jpg?w=1200&h=1200&auto=compress%2Cformat&fit=crop&dm=1654795233&s=5f137aa9a410a7ea3386c6972265111d"))

img_transforms = tvision.transforms.Compose([
    tvision.transforms.Resize((224, 224)),
    tvision.transforms.ToTensor()
])

clever_img_torch = img_transforms(image).unsqueeze(0)
clever_img_tf = tf.convert_to_tensor(image, dtype=tf.float32)
art_img_np = clever_img_torch.numpy()

In [20]:
print(torch.cuda.is_available())

True


## Directed Attacks

In [3]:
# helper function for getting probability, class id and class name
def convert2label(pred, is_clever=False):
    label_id, probability = 0, 0.
    
    if not is_clever:
        tensor_pred = torch.from_numpy(pred)
        probabilities = torch.nn.functional.softmax(tensor_pred, dim=1)
        label_id = int(torch.argmax(tensor_pred))
        probability = torch.max(probabilities)

    else:
        probability = torch.max(pred)
        label_id = int(torch.argmax(pred))
        
    return label_id, probability

In [4]:
# helper function for executing ART attacks
import time

def art_attack(attack, img, classifier, targeted=False, target=None):

    start = time.time()

    adv_img = attack.generate(x=img) if not targeted else attack.generate(x=img, y=target)

    normal_pred = classifier.predict(img)
    adv_pred = classifier.predict(adv_img)
    
    attack_time = time.time() - start

    return normal_pred, adv_pred, attack_time 


### Adversarial Robustness Toolbox

In [5]:
# prepare classifiers
mobile_classifier = PyTorchClassifier(
    model = mobile_model,
    loss = torch.nn.CrossEntropyLoss(),
    input_shape=(3,224, 224),
    nb_classes=len(imagenet_labels),
    optimizer=None,
    clip_values=(0,1)
)

vgg16_classifier = PyTorchClassifier(
    model = vgg16_model,
    loss = torch.nn.CrossEntropyLoss(),
    input_shape=(3,224,224),
    nb_classes=len(imagenet_labels),
    optimizer=None,
    clip_values=(0,1)
)

resnet_classifier = PyTorchClassifier(
    model = resnet_model,
    loss = torch.nn.CrossEntropyLoss(),
    input_shape=(3,224,224),
    nb_classes=len(imagenet_labels),
    optimizer=None,
    clip_values=(0,1)
)

#### Carlini Wagner L2

In [24]:
classifier_tab = [mobile_classifier, vgg16_classifier, resnet_classifier]
classifier_names = ["mobile_v3", "vgg16", "resnet50"]
iter = 5
norm_probs = np.zeros(iter)
norm_ids = np.zeros(iter)
adv_probs = np.zeros(iter)
adv_ids = np.zeros(iter)
times = np.zeros(iter)

target = np.zeros((1, len(imagenet_labels)))
target[0, 123] = 1


for classifier, name in zip(classifier_tab, classifier_names):
    
    attack = CarliniL2Method(classifier=classifier, targeted=True, learning_rate=0.1)
    
    for i in range(iter):
        norm, adv, att_time = art_attack(attack, img=art_img_np, classifier=classifier, targeted=True, target = target)
        norm_id, norm_prob = convert2label(norm)
        adv_id, adv_prob = convert2label(adv)
        
        times[i] = att_time
        norm_ids[i], norm_probs[i] = norm_id, norm_prob
        adv_ids[i], adv_probs[i] = adv_id, adv_prob
    
    if (not all(id == norm_ids[0] for id in norm_ids)) or (not all(id == adv_ids[0] for id in adv_ids)):
        print("ERROR: Not all ids in table are the same")
        break
    
    print("===================================================================================================================================================\n")
    print(f"Model: {name}, Correct class: {imagenet_labels[int(norm_ids[0])]}, Adversarial class: {imagenet_labels[int(adv_ids[0])]}\n")
    print(f"Average time: {np.average(times)}, Average normal probability: {np.average(norm_probs)}, Average adversary probability: {np.average(adv_probs)}")
    print("\n")
    

C&W L_2: 100%|██████████| 1/1 [00:07<00:00,  7.16s/it]



Model: mobile_v3, Correct class: wallaby, Adversarial class: spiny lobster

Average time: 7.391825532913208, Average normal probability: 0.24029135704040527, Average adversary probability: 0.012486599199473859




C&W L_2: 100%|██████████| 1/1 [00:13<00:00, 13.09s/it]



Model: vgg16, Correct class: dingo, Adversarial class: spiny lobster

Average time: 12.89978289604187, Average normal probability: 0.37113168835639954, Average adversary probability: 0.10962438434362412




C&W L_2: 100%|██████████| 1/1 [01:28<00:00, 88.71s/it]



Model: resnet50, Correct class: wallaby, Adversarial class: spiny lobster

Average time: 90.42976956367492, Average normal probability: 0.9998335838317871, Average adversary probability: 0.455068439245224




#### PGD

In [25]:
classifier_tab = [mobile_classifier, vgg16_classifier, resnet_classifier]
classifier_names = ["mobile_v3", "vgg16", "resnet50"]
iter = 5
norm_probs = np.zeros(iter)
norm_ids = np.zeros(iter)
adv_probs = np.zeros(iter)
adv_ids = np.zeros(iter)
times = np.zeros(iter)

target = np.zeros((1, len(imagenet_labels)))
target[0, 123] = 1


for classifier, name in zip(classifier_tab, classifier_names):
    
    attack = ProjectedGradientDescent(estimator=classifier, norm=np.inf, targeted=True)
    
    for i in range(iter):
        norm, adv, att_time = art_attack(attack, img=art_img_np, classifier=classifier, targeted=True, target = target)
        norm_id, norm_prob = convert2label(norm)
        adv_id, adv_prob = convert2label(adv)
        
        times[i] = att_time
        norm_ids[i], norm_probs[i] = norm_id, norm_prob
        adv_ids[i], adv_probs[i] = adv_id, adv_prob
    
    if (not all(id == norm_ids[0] for id in norm_ids)) or (not all(id == adv_ids[0] for id in adv_ids)):
        print("ERROR: Not all ids in table are the same")
        break
    
    print("===================================================================================================================================================\n")
    print(f"Model: {name}, Correct class: {imagenet_labels[int(norm_ids[0])]}, Adversarial class: {imagenet_labels[int(adv_ids[0])]}\n")
    print(f"Average time: {np.average(times)}, Average normal probability: {np.average(norm_probs)}, Average adversary probability: {np.average(adv_probs)}")
    print("\n")


Model: mobile_v3, Correct class: wallaby, Adversarial class: spiny lobster

Average time: 1.6023033618927003, Average normal probability: 0.24029135704040527, Average adversary probability: 0.9991568326950073





Model: vgg16, Correct class: dingo, Adversarial class: spiny lobster

Average time: 3.4947418689727785, Average normal probability: 0.37113168835639954, Average adversary probability: 0.8382400274276733





Model: resnet50, Correct class: wallaby, Adversarial class: spiny lobster

Average time: 24.27668514251709, Average normal probability: 0.9998335838317871, Average adversary probability: 1.0




#### Jacob Saliency Map Attack (JSMA)

In [26]:
classifier_tab = [mobile_classifier, vgg16_classifier, resnet_classifier]
classifier_names = ["mobile_v3", "vgg16", "resnet50"]
iter = 5
norm_probs = np.zeros(iter)
norm_ids = np.zeros(iter)
adv_probs = np.zeros(iter)
adv_ids = np.zeros(iter)
times = np.zeros(iter)

target = np.zeros((1, len(imagenet_labels)))
target[0, 123] = 1


for classifier, name in zip(classifier_tab, classifier_names):
    
    attack = SaliencyMapMethod(classifier=classifier, theta=0.3, gamma=0.05)
    
    for i in range(iter):
        norm, adv, att_time = art_attack(attack, img=art_img_np, classifier=classifier, targeted=True, target = target)
        norm_id, norm_prob = convert2label(norm)
        adv_id, adv_prob = convert2label(adv)
        
        times[i] = att_time
        norm_ids[i], norm_probs[i] = norm_id, norm_prob
        adv_ids[i], adv_probs[i] = adv_id, adv_prob
    
    if (not all(id == norm_ids[0] for id in norm_ids)) or (not all(id == adv_ids[0] for id in adv_ids)):
        print("ERROR: Not all ids in table are the same")
        break
    
    print("===================================================================================================================================================\n")
    print(f"Model: {name}, Correct class: {imagenet_labels[int(norm_ids[0])]}, Adversarial class: {imagenet_labels[int(adv_ids[0])]}\n")
    print(f"Average time: {np.average(times)}, Average normal probability: {np.average(norm_probs)}, Average adversary probability: {np.average(adv_probs)}")
    print("\n")

C:\Users\Wojciech\AppData\Local\Temp\ipykernel_3596\2078573611.py:8: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  adv_img = attack.generate(x=img) if not targeted else attack.generate(x=img, y=target)
JSMA: 100%|██████████| 1/1 [00:15<00:00, 15.62s/it]



Model: mobile_v3, Correct class: wallaby, Adversarial class: spiny lobster

Average time: 16.110242652893067, Average normal probability: 0.24029135704040527, Average adversary probability: 0.497028112411499




JSMA: 100%|██████████| 1/1 [04:02<00:00, 242.17s/it]



Model: vgg16, Correct class: dingo, Adversarial class: crayfish

Average time: 242.55334496498108, Average normal probability: 0.37113168835639954, Average adversary probability: 0.9981831908226013




JSMA: 100%|██████████| 1/1 [04:13<00:00, 253.49s/it]



Model: resnet50, Correct class: wallaby, Adversarial class: spiny lobster

Average time: 256.1481903076172, Average normal probability: 0.9998335838317871, Average adversary probability: 0.4923088550567627




### Non Targeted Attacks

In [27]:
def list_non_targeted_labels(label_ids):
    res = ""
    for i in range(len(label_ids)):
        res += imagenet_labels[int(label_ids[i])]
        if i != len(label_ids) - 1:
            res += ", "
    return res

#### ART

In [28]:
classifier_tab = [mobile_classifier, vgg16_classifier, resnet_classifier]
classifier_names = ["mobile_v3", "vgg16", "resnet50"]
iter = 5
norm_probs = np.zeros(iter)
norm_ids = np.zeros(iter)
adv_probs = np.zeros(iter)
adv_ids = np.zeros(iter)
times = np.zeros(iter)


for classifier, name in zip(classifier_tab, classifier_names):
    
    attack = DeepFool(classifier=classifier)
    
    for i in range(iter):
        norm, adv, att_time = art_attack(attack, img=art_img_np, classifier=classifier)
        norm_id, norm_prob = convert2label(norm)
        adv_id, adv_prob = convert2label(adv)
        
        times[i] = att_time
        norm_ids[i], norm_probs[i] = norm_id, norm_prob
        adv_ids[i], adv_probs[i] = adv_id, adv_prob
    
    print("===================================================================================================================================================\n")
    print(f"Model: {name}, Correct class: {imagenet_labels[int(norm_ids[0])]}, Adversarial class: {list_non_targeted_labels(adv_ids)}\n")
    print(f"Average time: {np.average(times)}, Average normal probability: {np.average(norm_probs)}, Average adversary probability: {np.average(adv_probs)}")
    print("\n")

DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

DeepFool: 100%|██████████| 1/1 [00:00<00:00,  3.45it/s]



Model: mobile_v3, Correct class: wallaby, Adversarial class: coyote, coyote, coyote, coyote, coyote

Average time: 0.3199635982513428, Average normal probability: 0.24029135704040527, Average adversary probability: 0.15137094259262085




DeepFool: 100%|██████████| 1/1 [00:00<00:00,  1.44it/s]



Model: vgg16, Correct class: dingo, Adversarial class: hare, hare, hare, hare, hare

Average time: 0.7221071720123291, Average normal probability: 0.37113168835639954, Average adversary probability: 0.9258731245994568




DeepFool: 100%|██████████| 1/1 [00:09<00:00,  9.67s/it]



Model: resnet50, Correct class: wallaby, Adversarial class: kit fox, kit fox, kit fox, kit fox, kit fox

Average time: 10.014681196212768, Average normal probability: 0.9998335838317871, Average adversary probability: 0.5609124898910522




In [29]:
mobile_model_double = DoubleModelTimmWrapper(mobile_model)
vgg16_model_double = DoubleModelTimmWrapper(vgg16_model)

mobile_classifier_double = PyTorchClassifier(
    model = mobile_model_double,
    loss = torch.nn.CrossEntropyLoss(),
    input_shape=(3,224,224),
    nb_classes=len(imagenet_labels),
    optimizer=None,
    clip_values=(0,1)
)

vgg16_classifier_double = PyTorchClassifier(
    model = vgg16_model_double,
    loss = torch.nn.CrossEntropyLoss(),
    input_shape=(3,224,224),
    nb_classes=len(imagenet_labels),
    optimizer=None,
    clip_values=(0,1)
)

classifier_tab = [mobile_classifier_double, resnet_classifier, vgg16_classifier_double]
classifier_names = ["mobile_v3", "resnet50", "vgg16"]
iter = 5
norm_probs = np.zeros(iter)
norm_ids = np.zeros(iter)
adv_probs = np.zeros(iter)
adv_ids = np.zeros(iter)
times = np.zeros(iter)


for classifier, name in zip(classifier_tab, classifier_names):
    
    attack = HopSkipJump(classifier=classifier, norm=2, max_iter=10)
    
    for i in range(iter):
        norm, adv, att_time = art_attack(attack, img=art_img_np, classifier=classifier)
        norm_id, norm_prob = convert2label(norm)
        adv_id, adv_prob = convert2label(adv)
        
        times[i] = att_time
        norm_ids[i], norm_probs[i] = norm_id, norm_prob
        adv_ids[i], adv_probs[i] = adv_id, adv_prob
    
    print("===================================================================================================================================================\n")
    print(f"Model: {name}, Correct class: {imagenet_labels[int(norm_ids[0])]}, Adversarial class: {list_non_targeted_labels(adv_ids)}\n")
    print(f"Average time: {np.average(times)}, Average normal probability: {np.average(norm_probs)}, Average adversary probability: {np.average(adv_probs)}")
    print("\n")

HopSkipJump: 100%|██████████| 1/1 [00:23<00:00, 23.43s/it]



Model: mobile_v3, Correct class: wallaby, Adversarial class: coyote, coyote, coyote, coyote, coyote

Average time: 23.6619300365448, Average normal probability: 0.23994162678718567, Average adversary probability: 0.18875771760940552




HopSkipJump: 100%|██████████| 1/1 [02:57<00:00, 177.03s/it]



Model: resnet50, Correct class: wallaby, Adversarial class: hare, hare, kit fox, kit fox, kit fox

Average time: 177.43666014671325, Average normal probability: 0.9998335838317871, Average adversary probability: 0.8303416728973388




HopSkipJump: 100%|██████████| 1/1 [11:11<00:00, 671.06s/it]



Model: vgg16, Correct class: dingo, Adversarial class: coyote, coyote, coyote, coyote, coyote

Average time: 696.7604658603668, Average normal probability: 0.3712729513645172, Average adversary probability: 0.252391105890274




In [30]:
classifier_tab = [mobile_classifier_double, vgg16_classifier_double, resnet_classifier]
classifier_names = ["mobile_v3", "vgg16", "resnet50"]
iter = 5
norm_probs = np.zeros(iter)
norm_ids = np.zeros(iter)
adv_probs = np.zeros(iter)
adv_ids = np.zeros(iter)
times = np.zeros(iter)


for classifier, name in zip(classifier_tab, classifier_names):
    
    attack = ElasticNet(classifier=classifier, max_iter=10)
    
    for i in range(iter):
        norm, adv, att_time = art_attack(attack, img=art_img_np, classifier=classifier)
        norm_id, norm_prob = convert2label(norm)
        adv_id, adv_prob = convert2label(adv)
        
        times[i] = att_time
        norm_ids[i], norm_probs[i] = norm_id, norm_prob
        adv_ids[i], adv_probs[i] = adv_id, adv_prob
    
    print("===================================================================================================================================================\n")
    print(f"Model: {name}, Correct class: {imagenet_labels[int(norm_ids[0])]}, Adversarial class: {list_non_targeted_labels(adv_ids)}\n")
    print(f"Average time: {np.average(times)}, Average normal probability: {np.average(norm_probs)}, Average adversary probability: {np.average(adv_probs)}")
    print("\n")

EAD:   0%|          | 0/1 [00:00<?, ?it/s]

EAD: 100%|██████████| 1/1 [00:06<00:00,  6.93s/it]



Model: mobile_v3, Correct class: wallaby, Adversarial class: coyote, coyote, coyote, coyote, coyote

Average time: 7.125292539596558, Average normal probability: 0.23994162678718567, Average adversary probability: 0.18650758266448975




EAD: 100%|██████████| 1/1 [03:46<00:00, 226.01s/it]



Model: vgg16, Correct class: dingo, Adversarial class: hare, hare, hare, hare, hare

Average time: 218.77918338775635, Average normal probability: 0.3712729513645172, Average adversary probability: 0.23584821820259094




EAD: 100%|██████████| 1/1 [01:06<00:00, 66.83s/it]



Model: resnet50, Correct class: wallaby, Adversarial class: kit fox, kit fox, kit fox, kit fox, kit fox

Average time: 67.75000014305115, Average normal probability: 0.9998335838317871, Average adversary probability: 0.4972759783267975




### Targeted Attacks

#### Cleverhans

##### Carlini Wagner L2

In [33]:
model_tab = [mobile_model, vgg16_model, resnet_model]
model_names = ["mobile_v3", "vgg16", "resnet50"]
iter = 5
norm_probs = np.zeros(iter)
norm_ids = np.zeros(iter)
adv_probs = np.zeros(iter)
adv_ids = np.zeros(iter)
times = np.zeros(iter)

image_cw2 = img_transforms(image).unsqueeze(0).cuda()

arr = np.zeros(1)
arr[0] = imagenet_labels.index('spiny lobster')
target_tensor = torch.from_numpy(arr).to(dtype=torch.long).cuda()
    
for model, name in zip(model_tab, model_names):
    
    for i in range(iter):

        start = time.time()
        adv_img = carlini_wagner_l2(model, image_cw2, n_classes=len(imagenet_labels), targeted=True, y=target_tensor, lr=0.1, max_iterations=10, binary_search_steps=10)

        with torch.no_grad():
            logits = model(image_cw2)
        norm = torch.nn.functional.softmax(logits, dim=1)    
        with torch.no_grad():
            logits = model(adv_img)
        adv = torch.nn.functional.softmax(logits, dim=1)

        end = time.time()

        norm_id, norm_prob = convert2label(norm, is_clever=True)
        adv_id, adv_prob = convert2label(adv, is_clever=True)
        
        times[i] = end - start
        norm_ids[i], norm_probs[i] = norm_id, norm_prob
        adv_ids[i], adv_probs[i] = adv_id, adv_prob
    
    if (not all(id == norm_ids[0] for id in norm_ids)) or (not all(id == adv_ids[0] for id in adv_ids)):
        print("ERROR: Not all ids in table are the same")
        break
    
    print("===================================================================================================================================================\n")
    print(f"Model: {name}, Correct class: {imagenet_labels[int(norm_ids[0])]}, Adversarial class: {imagenet_labels[int(adv_ids[0])]}\n")
    print(f"Average time: {np.average(times)}, Average normal probability: {np.average(norm_probs)}, Average adversary probability: {np.average(adv_probs)}")
    print("\n")


Model: mobile_v3, Correct class: wallaby, Adversarial class: spiny lobster

Average time: 1.574246883392334, Average normal probability: 0.24029138684272766, Average adversary probability: 0.28871172834187747



Model: vgg16, Correct class: dingo, Adversarial class: spiny lobster

Average time: 4.354140710830689, Average normal probability: 0.37113162875175476, Average adversary probability: 0.4082416519522667



Model: resnet50, Correct class: wallaby, Adversarial class: spiny lobster

Average time: 26.704101753234863, Average normal probability: 0.9998326247808695, Average adversary probability: 0.5135234698816326




##### PGD

In [10]:
model_tab = [mobile_model, vgg16_model, resnet_model]
model_names = ["mobile_v3", "vgg16", "resnet50"]
iter = 5
norm_probs = np.zeros(iter)
norm_ids = np.zeros(iter)
adv_probs = np.zeros(iter)
adv_ids = np.zeros(iter)
times = np.zeros(iter)

image_cw2 = img_transforms(image).unsqueeze(0).cuda()

arr = np.zeros(1)
arr[0] = imagenet_labels.index('spiny lobster')
target_tensor = torch.from_numpy(arr).to(dtype=torch.long).cuda()
    
for model, name in zip(model_tab, model_names):
    
    for i in range(iter):

        start = time.time()
        adv_img = projected_gradient_descent(model, image_cw2, norm=np.inf, eps=0.3, eps_iter=0.1, nb_iter=100, targeted=True, y=target_tensor)

        with torch.no_grad():
            logits = model(image_cw2)
        norm = torch.nn.functional.softmax(logits, dim=1)    
        with torch.no_grad():
            logits = model(adv_img)
        adv = torch.nn.functional.softmax(logits, dim=1)

        end = time.time()

        norm_id, norm_prob = convert2label(norm, is_clever=True)
        adv_id, adv_prob = convert2label(adv, is_clever=True)
        
        times[i] = end - start
        norm_ids[i], norm_probs[i] = norm_id, norm_prob
        adv_ids[i], adv_probs[i] = adv_id, adv_prob
    
    if (not all(id == norm_ids[0] for id in norm_ids)) or (not all(id == adv_ids[0] for id in adv_ids)):
        print("ERROR: Not all ids in table are the same")
        break
    
    print("===================================================================================================================================================\n")
    print(f"Model: {name}, Correct class: {imagenet_labels[int(norm_ids[0])]}, Adversarial class: {imagenet_labels[int(adv_ids[0])]}\n")
    print(f"Average time: {np.average(times)}, Average normal probability: {np.average(norm_probs)}, Average adversary probability: {np.average(adv_probs)}")
    print("\n")


Model: mobile_v3, Correct class: wallaby, Adversarial class: spiny lobster

Average time: 1.409954309463501, Average normal probability: 0.24029138684272766, Average adversary probability: 0.9984026432037354



Model: vgg16, Correct class: dingo, Adversarial class: spiny lobster

Average time: 3.6038485050201414, Average normal probability: 0.37113162875175476, Average adversary probability: 0.9023143529891968



Model: resnet50, Correct class: wallaby, Adversarial class: spiny lobster

Average time: 22.775072479248045, Average normal probability: 0.9998326247808695, Average adversary probability: 0.9999999996119161


